# Geração de texto básico

De inicio, sera usado o GPT 2 porque:

* É gratuito
* Podemos usar a biblioteca transformers

Obs: GPT 2 só entende inglês

#### Instalação das bibliotecas

##### Instalação do pytorch com suporte a gpu de cuda versão 12.8 (verificar cuda da placa de vídeo)
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu128
##### Instalação do transformers
!pip install transformers

In [2]:
# Importando Bibliotecas
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

In [3]:
# Inicializando o tokenizer e o modelo
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [13]:
# Encoding o prompt para coletar input ids
prompt = "Hello boss ..."
input_ids = tokenizer.encode(prompt, return_tensors="pt") # pt = pytorch
#input_ids # Visualização

# Gerar texto usando o modelo
outputs = model.generate(input_ids, max_length = 100)
#outputs # Visualização
tokenizer.decode(outputs[0], skip_special_tokens=True)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


"Hello boss ... I'm sorry, but I'm not sure if you're ready to go. I'm not sure if you're ready to go. I'm not sure if you're ready to go. I'm not sure if you're ready to go. I'm not sure if you're ready to go. I'm not sure if you're ready to go. I'm not sure if you're ready to go. I'm not sure if you're ready to go. I'm not sure"

In [14]:
def gerar_texto_simples(prompt, model, tokenizer, max_length = 100):
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    outputs = model.generate(input_ids, max_length = max_length)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [15]:
# Testando a função
prompt = "Hello my friend ..."
texto_gerado = gerar_texto_simples(prompt,
                                  model,
                                  tokenizer,
                                  max_length = 100)
print(texto_gerado)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Hello my friend ... I'm not going to be able to do this. I'm not going to be able to do this. I'm not going to be able to do this. I'm not going to be able to do this. I'm not going to be able to do this. I'm not going to be able to do this. I'm not going to be able to do this. I'm not going to be able to do this. I'm not going to be able to
